In [4]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
import string
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from numpy import savetxt

In [5]:
data_train=pd.read_csv("twitter_sentiment_analysis_train.csv")
data_test=pd.read_csv("twitter_sentiment_analysis_test.csv")

In [6]:
x_train=data_train["text"]
x_train=np.array(x_train)
y_train=data_train["airline_sentiment"]
y_train=np.array(y_train)
x_test=data_test["text"]
x_test=np.array(x_test)

In [7]:
from nltk.corpus import wordnet
def simple_pos_tag(tag):
    if tag.startswith("J"):
        return wordnet.ADJ
    elif tag.startswith("V"):
        return wordnet.VERB
    elif tag.startswith("N"):
        return wordnet.NOUN
    elif tag.startswith("R"):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [8]:
stop_words=stopwords.words("english")+list(string.punctuation)


In [9]:
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()



def NotNumber(w):
    for ch in w:
        if(ch>='0' and ch<='9'):
            return False
    return True

def clean(words):
    cleaned_words=[]
    words=word_tokenize(words)
    for word in words:
        if word not in stop_words and NotNumber(word)==True :
            tag=simple_pos_tag(pos_tag([word])[0][1])
            element=lemmatizer.lemmatize(word,pos=tag)
            cleaned_words.append(element)
    return cleaned_words
x_train=[clean(doc) for doc in x_train]            
            
x_train=[" ".join(doc) for doc in x_train]

In [10]:
x_test=[clean(doc) for doc in x_test]
x_test=[" ".join(doc) for doc in x_test]

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect=CountVectorizer(max_features=20000,max_df=0.3)
x_train=count_vect.fit_transform(x_train)
x_test=count_vect.transform(x_test)

In [12]:
from sklearn.linear_model import LogisticRegression
clf=LogisticRegression()
clf.fit(x_train,y_train)
y_pred=clf.predict(x_test)

C:\Users\deves\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [13]:
np.savetxt('twitter_sentiment_result.csv',y_pred,fmt='%s')